In [1]:
#%% ----------- IMPORT ------------------------
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [30]:
# da = pd.read_csv( "https://raw.githubusercontent.com/PrattSAVI/LIAVH/master/data/Artifacts.csv" , engine = 'python')
# ds = pd.read_csv( "https://raw.githubusercontent.com/PrattSAVI/LIAVH/master/data/Doorsils_and_Floor_Features_Level_Refs_Mackay.csv" , engine = 'python')

ds = pd.read_csv("/Users/saraeichner/Desktop/github/mohenjo-daro/data/Doorsils_and_Floor_Features_Level_Refs_Mackay.csv", encoding='unicode_escape')
da = pd.read_csv("/Users/saraeichner/Desktop/github/mohenjo-daro/data/Artifacts.csv",encoding='unicode_escape')

In [31]:
da.head()

,obj_plate_obj_ID,obj_plate,obj_plate_roman,obj_plate_num,obj_geo_B-U-S,obj_block,obj_house,obj_house_roman,obj_room,obj_geo_B-U-S_2,...,ptpb_paints_purple_black,ptpb_paints_red,ptpb_paints_white,ptpb_trimming,seal_type,seal_material,seal_dimensions_L_in,seal_dimensions_W_in,seal_dimensions_T_in,seal_dimensions_in_orig
0,074-001-X,74,LXXIV,1,14-X-01-04-X,14,1.0,I,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,074-002-X,74,LXXIV,2,13-X-02-07-X,13,2.0,II,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,074-003-X,74,LXXIV,3,13-X-03-19-X,13,3.0,III,19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,074-004-X,74,LXXIV,4,25-X-02-15-X,25,2.0,II,15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,074-005-X,74,LXXIV,5,18-X-00-110-X,18,0.0,NaN,110,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
da = da[['obj_plate_obj_ID','obj_block','obj_house','obj_room','obj_level_ft','obj_time_cat','obj_category','addl_type','addl_description']]
ds = ds[['Feature','Block','House','Room','Level_ft','Period_cited_in_text','Level_context','Text','Materials_notes']]
da.columns = ['Plate','Block','House','Room','Level_ft','Time_Cat','Feature','Type','Text']

ds['House'] = ds['House'].dropna(axis = 0)
ds['Block'] = ds['Block'].dropna(axis = 0)


In [33]:
ds['House'] = [r.split(', ')[0] for i,r in ds['House'].astype(str).iteritems()]

ds['N1'] = 'Floor Features'
da['N1'] = 'Artefacts'

display( ds.head(5) )
display( da.head(5))

/var/folders/v5/39j5vvc10pd4p0nqbxd7qv500000gn/T/ipykernel_79660/325653883.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  ds['House'] = [r.split(', ')[0] for i,r in ds['House'].astype(str).iteritems()]


,Feature,Block,House,Room,Level_ft,Period_cited_in_text,Level_context,Text,Materials_notes,N1
0,Doorsill,4,00,79,-14,Intermediate I,Exact Measure,The only apparent entrances were two doors at ...,NaN,Floor Features
1,Doorsill,4,00,13,-18.2,Intermediate II,Exact Measure,The entrance to room 13 was moved a few feet t...,NaN,Floor Features
2,Doorsill,4,00,03,-18.4,Intermediate II,Exact Measure,"\nof the building, each tread in passage 3 to ...",NaN,Floor Features
3,Doorsill,4,00,NaN,-15.3,Intermediate II,Exact Measure,"In this phase, the eastern section (III) of th...",NaN,Floor Features
4,Door-Socket,4,00,NaN,NaN,Intermediate III,Raised Above Footing,Close to the western jamb of the doorway^ we f...,NaN,Floor Features


,Plate,Block,House,Room,Level_ft,Time_Cat,Feature,Type,Text,N1
0,074-001-X,14,1.0,4,-8.2,Late III,Animals,Model animal,"Pottery, without a slip. This bird is probably...",Artefacts
1,074-002-X,13,2.0,7,-9.2,Late III,Animals,Model animal,Pottery; no slip. A fowl or dove on a narrow p...,Artefacts
2,074-003-X,13,3.0,19,-6,Late II,Animals,Model animal,"Pottery, with no slip. A brid's head and neck,...",Artefacts
3,074-004-X,25,2.0,15,-8.4,Late III,Animals,Model animal,"Pottery, with a red slip. A dove, as shown by ...",Artefacts
4,074-005-X,18,0.0,110,-5.8,Late II,Animals,Model animal,"Pottery, with traces of a fine white slip whic...",Artefacts


WATER FEATURES 

In [34]:
# dw = pd.read_csv( "https://raw.githubusercontent.com/PrattSAVI/LIAVH/master/data/Wells_and_Water_Features_Refs_Mackay.csv" , engine = 'python' )
dw = pd.read_csv("/Users/saraeichner/Desktop/github/mohenjo-daro/data/Wells_and_Water_Features_Refs_Mackay.csv",encoding='unicode_escape')

In [35]:
dw = dw.dropna( axis = 0 , how = 'all')
dw = dw.drop( ['Size_notes','Page_num' ], axis = 1 )

dw = dw[['Feature','Block','House','Room','Level_ft','Period_cited_in_text','Text','Plate_num']].copy()
dw.columns = ['Feature','Block','House','Room','Level_ft' , 'Time_Cat' , 'Text' ,'Plate']

#If Block/ House is not defined, Remove from list
dw = dw[~pd.isna(dw['Block']) ]
dw = dw[~pd.isna(dw['House']) ]
dw['Block'] = dw['Block'].astype(int).astype(str)
dw['House'] = dw['House'].astype(int).astype(str)
dw['House'] = dw['House'].str.zfill(2)

#If time is NA, assign average value
dw.loc[ (pd.isna(dw['Level_ft'])) & (dw['Time_Cat']=='Late  II and I') , 'Level_ft'] = -7
dw.loc[ (pd.isna(dw['Level_ft'])) & (dw['Time_Cat']=='Late III') , 'Level_ft'] = -9.9
dw.loc[ (pd.isna(dw['Level_ft'])) & (dw['Time_Cat']=='Intermediate I') , 'Level_ft'] = -13
dw.loc[ (pd.isna(dw['Level_ft'])) & (dw['Time_Cat']=='Intermediate II') , 'Level_ft'] = -15.9
dw.loc[ (pd.isna(dw['Level_ft'])) & (dw['Time_Cat']=='Intermediate III') , 'Level_ft'] = -20.4

dw['N1'] = 'Water Features'

dw.sample(5)

,Feature,Block,House,Room,Level_ft,Time_Cat,Text,Plate,N1
57,Drain,9,06,79,-7.0,Late II and I,The north-western corner of room 37 was twice ...,NaN,Water Features
44,Pavement,9,09,NaN,-9.9,Late III,The wash-down pavement in the north-eastern co...,NaN,Water Features
37,Well,7,08,19,-13.0,Intermediate I,No trace of any paving of this period was foun...,NaN,Water Features
4,Well,7,06,75,-20.4,Intermediate III,The well (75) must have been in use at this pe...,NaN,Water Features
53,Pavement,7,05,NaN,-11.6,Late III,In House V a large expanse of pavement round t...,NaN,Water Features


MERGE

In [36]:
df = da.append( ds )
df = df.append( dw )

df[ df['N1'] == 'Water Features' ].sample(5)

/var/folders/v5/39j5vvc10pd4p0nqbxd7qv500000gn/T/ipykernel_79660/1476502166.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = da.append( ds )
/var/folders/v5/39j5vvc10pd4p0nqbxd7qv500000gn/T/ipykernel_79660/1476502166.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append( dw )


,Plate,Block,House,Room,Level_ft,Time_Cat,Feature,Type,Text,N1,Period_cited_in_text,Level_context,Materials_notes
21,"PI. XLI, c",10,01,NaN,-15.9,Intermediate II,Chute,NaN,"The water escaped through an aperture, 3.75 in...",Water Features,NaN,NaN,NaN
33,NaN,9,06,NaN,-13.0,Intermediate I,Pavement,NaN,"A pavement, some 6 ft sq at the eastern end of...",Water Features,NaN,NaN,NaN
46,NaN,9,01,NaN,-9.9,Late III,Pottery - Jar,NaN,This pavement is 10.7 ft. below datum and a po...,Water Features,NaN,NaN,NaN
1,NaN,7,04,79,-18.0,Intermediate III,Pavement,NaN,The little patch of paving (79) at the level ...,Water Features,NaN,NaN,NaN
4,NaN,7,06,75,-20.4,Intermediate III,Well,NaN,The well (75) must have been in use at this pe...,Water Features,NaN,NaN,NaN


In [42]:
df.head()

,Plate,Block,House,Room,Level_ft,Time_Cat,Feature,Type,Text,N1,Period_cited_in_text,Level_context,Materials_notes
0,074-001-X,14,1.0,4,-8.2,Late III,Animals,Model animal,"Pottery, without a slip. This bird is probably...",Artefacts,NaN,NaN,NaN
1,074-002-X,13,2.0,7,-9.2,Late III,Animals,Model animal,Pottery; no slip. A fowl or dove on a narrow p...,Artefacts,NaN,NaN,NaN
2,074-003-X,13,3.0,19,-6,Late II,Animals,Model animal,"Pottery, with no slip. A brid's head and neck,...",Artefacts,NaN,NaN,NaN
3,074-004-X,25,2.0,15,-8.4,Late III,Animals,Model animal,"Pottery, with a red slip. A dove, as shown by ...",Artefacts,NaN,NaN,NaN
4,074-005-X,18,0.0,110,-5.8,Late II,Animals,Model animal,"Pottery, with traces of a fine white slip whic...",Artefacts,NaN,NaN,NaN


## stuck here with duplicate labels that I don't see

In [44]:
df['Block'] = df['Block'].astype(str) 

df['House'] = df['House'].replace(np.nan,None)
df['House'] = df['House'].replace('nan',None)
#df['House'] = df['House'].dropna(axis = 0)
#df['House'] = df['House'].astype(int).astype(str).str.zfill(2)
df['House'] = df['House'].astype(str).str.zfill(2)

#Filter Data to Selected Houses and Rooms
#df = df[ df['Block'].isin([ '4','7','9','10' ]) ]
#df = df[ df['House'].isin( ['07', '04', '03', '05', '06'] ) ]

#Filter Data to Selected Houses and Rooms
df = df[ df['Block'].isin([ '7']) ]
df = df[ df['House'].isin( ['03', '04', '05'] ) ]



In [45]:
df = df[ df.Level_ft != '?' ]
df = df[ df.Level_ft != '[?]' ]

df.loc[ df['Level_ft'] == 'SURFACE' , 'Level_ft' ] = 0
df.loc[ df['Level_ft'] == 'Surface' , 'Level_ft' ] = 0

df['Level_ft'] = df['Level_ft'].astype(float)

domestic = ['Animals', 'Cooking Baking Utensils', 'Personal Ornament', 
            'Vessels','Games And Toys', 'Human Figure', 'Household Objects'
            'Pottery - Incised and Painted', 'Pottery - Plain and Banded','Pottery - Incised and Painted - Upper']
craft = [ 'Tools', 'Lithics' , 'Metal']
trade = ['Weights And Measures', 'Beads', 'Seal' ]

well = [ 'Well', 'Well Steening', 'Well Feature Detail', 'Well - Pit']
drain = ['Pottery - Jar','Pipe', 'Drain','Outfall', 'Channel', 'Pit', 'Cesspit','Niche','Aperture', 'Chute', 'Drain Below Stair']
platform = ['Pavement']

df['Class'] = None

#Floor Features
df.loc[df['Feature'].isin(domestic) , 'Class'] = 'Domestic'
df.loc[df['Feature'].isin(craft) , 'Class'] = 'Craft'
df.loc[df['Feature'].isin(trade) , 'Class'] = 'Trade'

#Water Class
df.loc[df['Feature'].isin(well) , 'Class'] = 'Well'
df.loc[df['Feature'].isin(drain) , 'Class'] = 'Drainage'
df.loc[df['Feature'].isin(platform) , 'Class'] = 'Water Platforms'

#Catch All Else
df.loc[ (df['N1']=='Artefacts') & (pd.isna(df['Class'])) , 'Class'] = 'Other'

In [46]:
def div_text( df , col ):  #Style Text by dividing at 6th word with <br>
    texts = []
    for i,r in df.iterrows():
        if r[col]:
            span = 6
            words = str(r[col]).split(" ")
            res = [" ".join(words[i:i+span]) for i in range(0, len(words), span)]
            res = "<br>".join( res )
            texts.append( res )
        else:
            texts.append( None )
    return texts

df['Text2'] = div_text( df , 'Text' )
df['Text2'] = df['Text2'].replace('nan' , 'Not Available' )

#df.sample(2)

In [47]:
df.head()

,Plate,Block,House,Room,Level_ft,Time_Cat,Feature,Type,Text,N1,Period_cited_in_text,Level_context,Materials_notes,Class,Text2
20,NaN,7,03,NaN,-7.7,NaN,Foundation,NaN,Mlost of the walls of Houses III and IV rested...,Floor Features,Late II,Average,burnt brick,None,Mlost of the walls of Houses<br>III and IV res...
21,NaN,7,03,"41, 44, 48",-6.1,NaN,Foundation,NaN,"This building was of Late lb date, for the fou...",Floor Features,Late Ib,NaN,NaN,None,"This building was of Late lb<br>date, for the ..."
22,NaN,7,04,72,-13.9,NaN,Niche,NaN,House IV which was fairly complete was again s...,Floor Features,Intermediate I OR Intermediate II,Average,NaN,Drainage,House IV which was fairly complete<br>was agai...
23,NaN,7,04,NaN,-21.7,NaN,Doorsill,NaN,The badly ruined house (IV) which was entered ...,Floor Features,Intermediate III,General Measure,NaN,None,The badly ruined house (IV) which<br>was enter...
24,NaN,7,05,NaN,-13.6,NaN,Pavement,NaN,Against the eastern part of the southern wall ...,Floor Features,Intermediate I,Exact Measure,unusually fine pavement of two courses of bricks,Water Platforms,Against the eastern part of the<br>southern wa...


PLOTLY

In [48]:
time_order = [ 'Late Ia','Late Ib','Late II','Late III',
 'Intermediate I','Intermediate II','Intermediate III',
 'Early Periods']

ff = go.Figure() #Start an empty Figure Object

In [67]:
#-----DRAW ==> STRATA LINES
for t in time_order: #Draw the Strata Lines
    at = df[ df['Time_Cat'] == t ]
    mins = at[['Block','House','Level_ft']].groupby( by = ['Block','House'] ).min().reset_index()
    maxs = at[['Block','House','Level_ft']].groupby( by = ['Block','House'] ).max().reset_index()
    al = mins.join( maxs.set_index('Block','House') , on = ['Block','House'] , rsuffix = '_max')
     
    ff.add_trace(
        go.Scatter( #lines
                x=[al['Block'],al['House']], 
                y=al['Level_ft_max'],
                fill=None,
                mode='lines',
                line=dict(width=0),
                showlegend = False
                )
            )
    ff.add_trace(
        go.Scatter( #White Fill
                x=[ al['Block'],al['House'] ],
                y=al['Level_ft'],
                fill='tonexty',
                mode='lines',
                fillcolor='rgba(255,255,255,0.2)',
                line=dict(width=0, color='white'),
                showlegend = False,
                )
            )

/var/folders/v5/39j5vvc10pd4p0nqbxd7qv500000gn/T/ipykernel_79660/1584992312.py:6: FutureWarning:

In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.



ValueError: len(left_on) must equal the number of levels in the index of "right"

In [65]:
#------DRAW ==> FLOOR FEATURE
cmap = plt.cm.get_cmap('Set2')
colors = [ 'rgb(' + str(int(cmap(i)[0]*255)) + ',' + str(int(cmap(i)[1]*255)) + ',' + str(int(cmap(i)[2]*255)) + ')' for i in np.linspace( 0 ,1 , len( df[df['N1']=='Floor Features']['Feature'].unique() ) ) ]
count = 0
for f in df[df['N1']=='Floor Features']['Feature'].unique(): # Draw Floor Features
    dff = df[ (df['N1']=='Floor Features') & (df['Feature'] == f) ]

    ff.add_trace( # Draw Floor Features
        go.Scatter(
            mode='markers',
            x= [ dff['Block'] , dff['House'] ],
            y = dff['Level_ft'],
            legendgroup =  f,
            text = '<b>' + dff['Feature'] + '</b><br>'+ dff['Text2'],
            hovertemplate = "%{text}<extra></extra>",
            marker_symbol = 'line-ew',
            marker_line_color = colors[count], 
            marker_line_width = 2.5, marker_size = 25,
            name = f
            )
        )
    count = count + 1

In [62]:
#------DRAW ==> WATER FEATURE
cmap = plt.cm.get_cmap('Set1')
colors = [ 'rgb(' + str(int(cmap(i)[0]*255)) + ',' + str(int(cmap(i)[1]*255)) + ',' + str(int(cmap(i)[2]*255)) + ')' for i in np.linspace( 0 ,1 , len( df[df['N1']=='Water Features']['Feature'].unique() ) ) ]

count = 0
for f in df[df['N1']=='Water Features']['Class'].unique(): # Draw Floor Features
    dff = df[ (df['N1']=='Water Features') & (df['Class'] == f ) ]

    ff.add_trace( # Draw Water Features
        go.Scatter(
            mode='markers',
            x= [ dff['Block'] , dff['House'] ] ,
            y=dff['Level_ft'],
            legendgroup =  f,
            text = '<b>' + dff['Feature'] + '</b><br>'+ dff['Text2'],
            hovertemplate = "%{text}<extra></extra>",
            marker_symbol = 'y-up',
            marker_line_color= colors[count], 
            marker_line_width=13, marker_size=8,
            name = f,
            )
        )
    count = count + 1

In [63]:
#SWARM PLOT
def box_figure( df ): # Prepare the Swarm Plot

    cmap = plt.cm.get_cmap('Set1')
    colors = [ 'rgb(' + str(int(cmap(i)[0]*255)) + ',' + str(int(cmap(i)[1]*255)) + ',' + str(int(cmap(i)[2]*255)) + ')' for i in np.linspace( 0 ,1 , len( df['Class'].unique() ) ) ]
    count = 0
    
    df = df[df['N1'] == 'Artefacts']
    data_group = []
    for item in sorted( df['Class'].unique() ):
        data_group.append(
            {
                        'boxpoints': 'all',
                        'fillcolor': 'rgba(255,255,255,0)',
                        'hoveron': 'points',
                        'text': '<b>' + df[df['Class'] == item]['Feature'] + '</b>' + '<br>Desc: ' + df[df['Class'] == item]['Text2'] + '<br>Link: <a target="_blank" href=' + '>' + df[df['Class'] == item]['Plate'] + '</a>' ,
                        'hovertemplate': "%{text}" ,
                        'legendgroup': item ,
                        'line': {'color': 'rgba(255,255,255,0)'},
                        'marker': { 
                                    'color': colors[ count ] ,
                                    'size': 8,
                                    'opacity':0.8
                                    },
                        'name': item,
                        'pointpos':0,
                        'type': 'box',
                        'x': [ df[df['Class'] == item]['Block'].tolist() , df[df['Class'] == item]['House'].tolist()],
                        'y': df[df['Class'] == item]['Level_ft'].tolist(),
                        'jitter':0.5
                        })
        count = count+1

    return {
        'data': data_group,
    }
for d in box_figure(df)['data']: #Draw Point Plot
    ff.add_trace( go.Box( d ) )

In [68]:
#ADD NAME ANNOTATIONS
gr = df.groupby(by='Time_Cat').mean().reset_index()

ff.add_trace( # Add the Names
    go.Scatter(
        x = ['03']*len( gr ) ,
        y = gr['Level_ft'],
        mode = "text",
        text = '                  ' + gr['Time_Cat'].astype(str).str.upper() ,
        textposition = "middle right",
        showlegend = False,
        textfont = dict(
            family = 'Tisa Sans Pro',
            size = 8,
            color = 'grey'
        )
    ))

/var/folders/v5/39j5vvc10pd4p0nqbxd7qv500000gn/T/ipykernel_79660/2881685315.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [71]:
#LAYOUT
ff.update_layout( #Style Layout
    width = 600 , height = 800 ,
    hoverlabel=dict(
        bgcolor="white", 
        font_size=11, 
        font_family="Corbel"
    ),
    showlegend=True,
    dragmode = 'zoom',
    hovermode = 'closest',
    hoverdistance = 60,
    plot_bgcolor = '#ebebea',
    paper_bgcolor = '#ebebea',
    title = 'Block 7, Houses 3, 4 & 5 Artefact and Floor / Water Features',
    margin=dict(l=50,r=50,b=80,t=100, pad=25 ),
    xaxis = dict(
        gridcolor = "rgb(255, 255, 255,0.5)",
        showgrid = True),
    yaxis = dict(
        gridcolor = "rgb(255, 255, 255,0.5)",
        gridwidth = 0.1)
    )

ff.show()

In [72]:
# Write figure to HTML, for offline use.  
ff.write_html(r'/Users/saraeichner/Desktop/github/mohenjo-daro/112424_LIAVH_bl7_hs3-4-5_plot.html')